In [1]:
import pandas as pd

In [2]:
dados = pd.read_csv("vgsales.csv")

In [3]:
#remove valores NaN
dados = dados.dropna()

In [4]:
#obtendo apenas variaveis desejadas
#desjamos prever a venda de um videogame na america do norte em função da plataforma, ano, genero e autora
dataset = dados.loc[:, ["Platform", "Year", "Genre" , "Publisher", "NA_Sales"]]

In [5]:
dataset

,Platform,Year,Genre,Publisher,NA_Sales
0,Wii,2006.0,Sports,Nintendo,41.49
1,NES,1985.0,Platform,Nintendo,29.08
2,Wii,2008.0,Racing,Nintendo,15.85
3,Wii,2009.0,Sports,Nintendo,15.75
4,GB,1996.0,Role-Playing,Nintendo,11.27
...,...,...,...,...,...
16593,GBA,2002.0,Platform,Kemco,0.01
16594,GC,2003.0,Shooter,Infogrames,0.01
16595,PS2,2008.0,Racing,Activision,0.00
16596,DS,2010.0,Puzzle,7G//AMES,0.00


In [6]:
relevantes = (dataset["Platform"] == "PS4") | (dataset["Platform"] == "Wii") |  (dataset["Platform"] == "XOne")
dataset = dataset.loc[relevantes,:]

In [8]:
top_10 = dataset["Publisher"].value_counts().head(10)
top_10_dict = top_10.to_dict()
chav = top_10_dict.keys()


keys = []


for i in chav:
    keys.append(i)

['Ubisoft',
 'Activision',
 'Electronic Arts',
 'Namco Bandai Games',
 'Nintendo',
 'THQ',
 'Konami Digital Entertainment',
 'Take-Two Interactive',
 'Sega',
 'Warner Bros. Interactive Entertainment']